# Single-voxel simulations

Monte-Carlo simulations were performed for a single-voxel representative of white-matter, to evaluate the impact of each subsampling method (SC vs TRUNC) on the estimation of diffusion parameters derived from DTI (i.e., FA, MD, AD, and RD maps) and DKI (i.e., MK, AK, and RK maps). We simulated the diffusion-weighted signal under different background noise conditions (SNR=10, 20, 30, 40, 50).  

In [10]:
%pip install dipy

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import sys
import time
from pathlib import Path
from tqdm import tqdm

# Parallelization
import multiprocessing

# Export data
import csv

# module to load nifti images
import nibabel as nib
import numpy as np

# fucntion to construct gradient table and design matrix
from dipy.core.gradients import gradient_table
from dipy.core.sphere import HemiSphere, disperse_charges

#from dipy.data import fetch_stanford_hardi, read_stanford_hardi
from dipy.io.gradients import read_bvals_bvecs
from dipy.data import (get_fnames, get_sphere)

# function to compute quadratic form of tensor from evals and evecs
from dipy.reconst.dki import decompose_tensor, Wrotate

# fucntion to add noise to simulated data
from dipy.sims.voxel import dki_signal, multi_tensor_dki, _check_directions, all_tensor_evecs


In [4]:
def dki_fitting(info):
    np.random.seed(None)
    noisy = dki_signal(
        info[1],
        info[2], 
        info[3], 
        S0=info[4],
        snr=info[5],
    )
    return [info[0], noisy]

In [5]:
# ------------------------------------------------------------------------------
# Defining working directories:
# ------------------------------------------------------------------------------
cwd = Path.cwd()
path_sims = cwd
path_sc = path_sims / "03-anisotropic_voxel/sc"
path_trunc = path_sims / "03-anisotropic_voxel/trunc"
path_to_save = path_sims / "05-sims_signal-nifti/independent"

In [6]:
# ------------------------------------------------------------------------------
# Defining simulation parameters
# ------------------------------------------------------------------------------
# The experiment is repeated for 100 diffusion tensor directions and 100 noise repeats
nReps = 100  # number of noise instances
nDTdirs = 100  # number of tensor directions
SNR = [10, 20, 30, 40, 50, 1000]  # noise levels
subsets = ["full", "subset5", "subset10", "subset20", "subset30", "subset40", "subset50"] 
methods = ["sc", "trunc"] # two different subsampling methods

# number of activated cores for parallel processing:
n_cores = 12

In [7]:
# ------------------------------------------------------------------------------
# Ground truth parameters:
# ------------------------------------------------------------------------------
# Reference for GT parameters: shorturl.at/EIQV6 - Section 3.2.1.

# ------------------------------------------------------------------------------
# 1. Simulating diffusion tensor
# ------------------------------------------------------------------------------
DT0 = np.zeros((3, 3))

DT0[0][0] = 0.1783e-3
DT0[0][1] = 0.0011e-3
DT0[0][2] = 0.0125e-3
DT0[1][0] = 0.0011e-3
DT0[1][1] = 0.1459e-3
DT0[1][2] = 0.0034e-3
DT0[2][0] = 0.0125e-3
DT0[2][1] = 0.0034e-3
DT0[2][2] = 0.4028e-3

dt = np.array([DT0[0][0], DT0[0][1], DT0[1][1], DT0[0][2], DT0[1][2], DT0[2][2]])

# ------------------------------------------------------------------------------
# 2. Simulating kurtosis tensor
# ------------------------------------------------------------------------------
kt0 = np.zeros((15))

kt0[0] = 0.5698
kt0[1] = 0.3208
kt0[2] = 2.6049
kt0[3] = -0.1142
kt0[4] = -1.1531
kt0[5] = 0.3944
kt0[6] = -0.5409
kt0[7] = 0.7220
kt0[8] = 0.777
kt0[9] = 0.2865
kt0[10] = 0.4700
kt0[11] = 0.2065
kt0[12] = -0.0005
kt0[13] = 0.1655
kt0[14] = -0.4157

# ------------------------------------------------------------------------------
# 3. Simulating S0 signal
# ------------------------------------------------------------------------------
# Parameters that define the unweighted signal at 3T (assuming no T1 relaxation):

# T2 relaxation (ms)
T2_tissue = 80  # Wansapura et al., 1999

# Proton density (percentage units)
PD_tissue = 70  # Abbas et al., 2015

# TE (ms) 
TE = 89 # MIG_N2Treat project protocol

# Assuming no T1 relaxation, the non-weighted signal for a voxel that has only tisssue or only water:
k = 10
S0_sims = k * PD_tissue * np.exp(-TE / T2_tissue)

print('S0 signal for a voxel with only tissue is ' + str(S0_sims))

S0 signal for a voxel with only tissue is 230.11526488059488


In [8]:
# ------------------------------------------------------------------------------
# Simulate tensor rotations:
# ------------------------------------------------------------------------------
theta = np.pi * np.random.rand(nDTdirs)
phi = 2 * np.pi * np.random.rand(nDTdirs)
hsph_initial = HemiSphere(theta=theta, phi=phi)
hsph_updated, potential = disperse_charges(hsph_initial, 5000)
DTdirs = hsph_updated.vertices

In [9]:
# ------------------------------------------------------------------------------
print("Generating single-voxel DWIs...")
# ------------------------------------------------------------------------------

for m_i, method in enumerate(methods):
    print("Processing " + method + "---------------------------------")

    if method == "sc":
        path_out=path_sc
    else:
        path_out=path_trunc

    for snr_i, snr in enumerate(SNR):
        print("Processing snr:" + str(snr) + "---------------------------------")
        
        for s_i, subset in enumerate(subsets):
            print("Processing " + subset + "---------------------------------")

            bv1 = subset + "_" + method + "_b0b1000b2000.bvec"
            bv2 = subset + "_" + method + "_b0b1000b2000.bval"

            fbvecs = os.path.join(path_out, bv1)
            fbvals = os.path.join(path_out, bv2)

            bvals, bvecs = read_bvals_bvecs(str(fbvals), str(fbvecs))
            gtab_sims = gradient_table(bvals, bvecs, b0_threshold=0)
            
            DWIs = np.zeros((nDTdirs * nReps, bvals.size))

            for d_i in tqdm(np.arange(nDTdirs)):

                time.sleep(0.3)
                # for current simulated tensor direction
                angles = DTdirs[d_i]
                sticks = _check_directions(angles)

                R = all_tensor_evecs(sticks)
                DT = np.dot(np.dot(R, DT0), R.T)
        
                eigvals, eigvects = decompose_tensor(DT)

                dt_rot = np.array([DT[0][0], DT[0][1], DT[1][1], DT[0][2], DT[1][2], DT[2][2]])

                kt_rot = Wrotate(kt0, R.T)

                input_list = []
                for n_i in np.arange(d_i * nReps, (d_i + 1) * nReps, dtype=int):
                    # Fitting DKI model to estimate the tensor parameters:
                    slice_info = [n_i, gtab_sims, dt_rot, kt_rot, S0_sims, snr]
                    input_list.append(slice_info)

                if n_cores is None:
                    n_cores = multiprocessing.cpu_count()

                fitpool = multiprocessing.Pool(
                    processes=n_cores)  # Create parallel processes
                fitpool_pids_initial = [proc.pid for proc in fitpool._pool]  # Get initial process identifications (PIDs)
                fitresults = fitpool.map_async(dki_fitting, input_list)  # Give jobs to the parallel processes

                # Busy-waiting: until work is done, check whether any worker dies (in that case, PIDs would change!)
                while not fitresults.ready():
                    fitpool_pids_new = [
                        proc.pid for proc in fitpool._pool
                    ]  # Get process IDs again
                    if (
                        fitpool_pids_new != fitpool_pids_initial
                    ):  # Check whether the IDs have changed from the initial values
                        print(
                            ""
                        )  # Yes, they changed: at least one worker has died! Exit with error
                        print(
                            "ERROR: some processes died during parallel fitting. Exiting with 1."
                        )
                        print("")
                        sys.exit(1)

                # Work done: get results
                fitlist = fitresults.get()

                
                # Collect fitting output and re-assemble MRI slices
                for fit_i, fit_rep in enumerate(fitlist):
                    DWIs[fit_rep[0], :] = fit_rep[1]
                        
            converted_array = np.asfarray(DWIs, dtype=np.float32)

            nifti_array = converted_array.reshape((2, 6, 12, bvals.size))
            nifti_file = nib.Nifti2Image(nifti_array, affine=np.eye(4))
            
            out_dir = os.path.join(path_to_save, 'snr{}'.format(snr), subset)
            if not os.path.exists(out_dir):
                os.makedirs(out_dir)
                
            nib.save(nifti_file, os.path.join(out_dir, 'dwi_sims_snr{}_{}_method-{}.nii'.format(snr, subset, method)))

Generating single-voxel DWIs...
Processing sc---------------------------------
Processing snr:10---------------------------------
Processing full---------------------------------


 35%|███▌      | 35/100 [00:24<00:45,  1.42it/s]Process ForkPoolWorker-432:
Process ForkPoolWorker-430:
Process ForkPoolWorker-422:
Process ForkPoolWorker-425:
Process ForkPoolWorker-427:
Process ForkPoolWorker-421:
Process ForkPoolWorker-424:
Process ForkPoolWorker-426:
Process ForkPoolWorker-431:
Process ForkPoolWorker-429:
Process ForkPoolWorker-428:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Process ForkPoolWorker-423:
  File "/usr/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call

KeyboardInterrupt: 